In [1]:
# Create a tool(a function really)
# Put that into a list of tools

from langchain_core.tools import tool

@tool
def get_weather(city: str) -> str:
    """
    Returns the weather for a given city
    """
    return f'The weather in the {city} is 28 degrees celsius and warm.'

tools = [get_weather]
tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

In [2]:
#Create a llm object and a prompt template

from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate

llm = OllamaLLM(model='mistral', temperature=0)


prompt_template = PromptTemplate.from_template("""
You are an AI assistant that can call tools.

Here are the available tools:
{tool_descriptions}

When needed, respond with a JSON object in the following format:

{format_instructions}

User input:
{text}

Remember: Only output the JSON. Do not explain.
""")

In [3]:
# create a ToolCall pydantic class and use that in a JsonOutputParser

from pydantic import BaseModel
from langchain_core.output_parsers import JsonOutputParser

class ToolCall(BaseModel):
    tool: str
    args: dict

parser = JsonOutputParser(pydantic_object=ToolCall)

In [4]:
# Create a chain and arrange the inputs for the prompt

chain = prompt_template | llm | parser
user_input = "whats the weather like in Bangalore"

formatted_prompt = {
    "text": user_input,
    "tool_descriptions": tool_descriptions,
    "format_instructions": parser.get_format_instructions()
}

In [ ]:
# Invoke the chain, identify the tool to be called and run the tool and print the results
# In a more complete code, we would be sending back the tool results into the LLM context

try:
    tool_call = chain.invoke(formatted_prompt)
    print(f'Tool to call: {tool_call['tool']}')
    print(f'with arguments: {tool_call['args']}')
    
    for tool in tools:
        if tool.name == tool_call['tool']:
            result = tool.run(tool_call['args'])
            print(f'result: {result}')
            break
    
except Exception as e:
    print("Failed to parse or execute tool:", e)